#Vector Ensemble
Ensemble approach using Spark.

##SET DECADE (OR NOT)?

In [1]:
# if decade is set, then filter results accordingly.
decade = None
# decade = 1970

In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [3]:
## MLJ: Additional Extras
import time
import itertools
import json
import pickle

In [4]:
import os
# os.environ['PYSPARK_PYTHON'] = '/anaconda/bin/python'

In [5]:
import findspark
findspark.init()
print findspark.find()
# Depending on your setup you might have to change this line of code
#findspark makes sure I dont need the below on homebrew.
#os.environ['SPARK_HOME']="/usr/local/Cellar/apache-spark/1.5.1/libexec/"
#the below actually broke my spark, so I removed it. 
#Depending on how you started the notebook, you might need it.
# os.environ['PYSPARK_SUBMIT_ARGS']="--master local pyspark --executor-memory 4g"

/home/vagrant/spark


In [6]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local[4]')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)

In [7]:
sc._conf.getAll()

[(u'spark.executor.memory', u'2g'),
 (u'spark.master', u'local[4]'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.driver.memory', u'8g'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.app.name', u'pyspark')]

In [8]:
import sys
rdd = sc.parallelize(xrange(2),2)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]',
 '2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]']

In [9]:
sys.version

'2.7.10 |Anaconda 2.3.0 (64-bit)| (default, May 28 2015, 17:02:03) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [10]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

##Setup Data For Pipeline
###Load and manipulate with Pandas

In [11]:
# load the lyrics from the approved "master" dataframe
lyrics_pd_df = pd.read_csv("../../data/conditioned/use-this-master-lyricsdf-extracted.csv")  

In [12]:
if decade:
    lyrics_pd_df = lyrics_pd_df[lyrics_pd_df['decade'] == decade]

##Add Labels for Data based on position
**Note: Spark ML seems picky about `label` being the column name**

In [13]:
# use positions for labeling
positions_10_percent = {
  10.0:range(1,11),
  20.0:range(11,21),
  30.0:range(21,31),
  40.0:range(31,41),
  50.0:range(41,51),
  60.0:range(51,61), 
  70.0:range(61,71),
  80.0:range(71,81),
  90.0:range(81,91),
  100.0:range(91,101)  
}

positions_25_percent = {
  25.0:range(1,26),
  50.0:range(26,51),
  75.0:range(51,76),
  100.0:range(76,101)
}

# binary classification for top 25
positions_top_25 = {
  0.0:range(1,26),
  1.0:range(26,101)
}

# binary classification for top 50
positions_top_50 = {
  0.0:range(1,51),
  1.0:range(51,101)
}

def labelForPosition(pos, positions=positions_top_50):
    for k,p in positions.iteritems():
        if pos in p:
            return k
    return None

In [14]:
# test label lookup
labelForPosition(1)

0.0

In [15]:
#label is position, e.g. 1-10, in this use
lyrics_pd_df['label'] = lyrics_pd_df.position.apply(labelForPosition)

In [16]:
lyrics_pd_df.sample(5).head()

,index,position,year,title.href,title,artist,lyrics,decade,song_key,lyrics_url,lyrics_abstract,label
767,767,68,1977,https://en.wikipedia.org/wiki/Cold_as_Ice_(For...,Cold as Ice,Foreigner,You're as cold as ice. You're willing to sacri...,1970,1977-68,http://lyrics.wikia.com/Foreigner:Cold_As_Ice,You're as cold as ice. You're willing to sacri...,1
3906,3906,7,2009,https://en.wikipedia.org/wiki/I%27m_Yours_(Jas...,I'm Yours,Jason Mraz,"Well, you dawned on me and you bet I felt it. ...",2000,2009-7,http://lyrics.wikia.com/Jason_Mraz:I%27m_Yours,"Well, you dawned on me and you bet I felt it. ...",0
3596,3596,97,2005,https://en.wikipedia.org/wiki/U_Already_Know,U Already Know,112,"uhh,112.uhh. Papa coming home, to give you som...",2000,2005-97,http://lyrics.wikia.com/112:U_Already_Know,"uhh,112.uhh. Papa coming home, to give you som...",1
4382,4382,83,2013,https://en.wikipedia.org/wiki/Body_Party,Body Party,Ciara,"My body is your party, baby. Nobody's invited ...",2010,2013-83,http://lyrics.wikia.com/Ciara:Body_Party,"My body is your party, baby. Nobody's invited ...",1
173,173,74,1971,https://en.wikipedia.org/wiki/When_You%27re_Ho...,"When You're Hot, You're Hot",Jerry Reed,Well me and Homer Jones and Big John Talley. H...,1970,1971-74,http://lyrics.wikia.com/Jerry_Reed:When_You%27...,Well me and Homer Jones and Big John Talley. H...,1


##Filter out Non-Lyric Records

In [17]:
# Check for nulls
empties = np.where(pd.isnull(lyrics_pd_df[['lyrics']]))
print "How many empties are there? {}".format(len(empties[0]))

How many empties are there? 103


In [18]:
# Check for instrumental mustic
instrumentals = np.where(lyrics_pd_df[['lyrics']].applymap(lambda x: x == 'Instrumental'))
print "How many instrumentals are there? {}".format(len(instrumentals[0]))

How many instrumentals are there? 20


In [19]:
# total non-lyrics
print "How many total non-lyrics are there? {}".format(len(empties[0]) + len(instrumentals[0]))

How many total non-lyrics are there? 123


In [20]:
lyrics_pd_df.shape

(4500, 12)

In [21]:
# filter out null lyrics
lyrics_pd_df = lyrics_pd_df.dropna(axis=0, how='any', thresh=None, subset=['lyrics'], inplace=False)

In [22]:
lyrics_pd_df.shape

(4397, 12)

In [23]:
# filter out instrumentals
lyrics_pd_df = lyrics_pd_df[lyrics_pd_df['lyrics'] != 'Instrumental']

In [24]:
lyrics_pd_df.shape

(4377, 12)

###Load Corpus and Vocab(s) As Needed
**
NOTES:
* These files need to be locally unzipped, e.g. the `corpus_vocabs.7z` under `data` folder
* The unzipped files should not be added back to GitHub, they are excluded in .gitignore
* It is assumed that the .7z are unzipped into their own folder
**

In [25]:
buildFilename = (lambda name, decade, ext : name + str(decade) + "." + ext if decade else name + "." + ext) 

In [26]:
# base_path will get manipulated based on decade or not
base_path = "../../data/conditioned/"
if decade:
    base_path = base_path + "decades/" + str(decade) + "/"
else:
    base_path = base_path + "corpus_vocabs/"

print "Will be using base_path --> ",base_path

Will be using base_path -->  ../../data/conditioned/corpus_vocabs/


In [27]:
## with base path set, load up needed resources

corpus = None # this is a pickle file
adjid2word = None
nounid2word = None

# corpus
try:   
    corpus = pickle.load( open(base_path + buildFilename("corpus",decade,"p") , "rb" ) ) 
except Exception as e:
    print "unable to find corpus{} in directory '{}' (have you unzipped vocabs to a directory?), msg --> {}".format((str(decade) if decade else ""),base_path,e)

# adjid2word
try:
    with open(base_path + buildFilename("adjid2word",decade,"json"), 'r') as fp:
        adjid2word = json.load(fp)
except Exception as e:
    print "unable to find adjid2word{} in directory '{}' (have you unzipped vocabs to a directory?), msg --> {}".format((str(decade) if decade else ""),base_path,e)

# nounid2word
try:
    with open(base_path + buildFilename("nounid2word",decade,"json"), 'r') as fp:
        nounid2word = json.load(fp)
except Exception as e:
    print "unable to find nounid2word{} in directory '{}' (have you unzipped vocabs to a directory?), msg --> {}".format((str(decade) if decade else ""),base_path,e)    

In [28]:
print "How big is corpus? ", len(corpus)
print "How big is adjid2word? ", len(adjid2word)
print "How big is nounid2word? ", len(nounid2word)

How big is corpus?  35472
How big is adjid2word?  3379
How big is nounid2word?  5144


In [29]:
corpus[:2]

[[(5139, 1)], [(135, 1), (1887, 1)]]

In [30]:
adjid2word['0'] # keys are index over size of vocab

u'suicidal'

In [31]:
nounid2word['0'] # keys are index over size of vocab

u'jockin'

## Exploration: Alter `corpus` to use column `position` as a label
Idea: take a percentile of data base on position range (1-10, 11-20, etc) to label where songs fall in the charts.

###Convert and manipulate with Spark

In [32]:
# convert from pandas to spark dataframe
lyricsdf = sqlsc.createDataFrame(lyrics_pd_df)

In [33]:
lyricsdf.show(3)

+-----+--------+----+--------------------+--------------------+-------------------+--------------------+------+--------+--------------------+--------------------+-----+
|index|position|year|          title.href|               title|             artist|              lyrics|decade|song_key|          lyrics_url|     lyrics_abstract|label|
+-----+--------+----+--------------------+--------------------+-------------------+--------------------+------+--------+--------------------+--------------------+-----+
|    0|       1|1970|https://en.wikipe...|Bridge over Troub...|Simon and Garfunkel|When you're weary...|  1970|  1970-1|http://lyrics.wik...|When you're weary...|  0.0|
|    1|       2|1970|https://en.wikipe...|(They Long to Be)...|     The Carpenters|Why do birds sudd...|  1970|  1970-2|http://lyrics.wik...|Why do birds sudd...|  0.0|
|    2|       3|1970|https://en.wikipe...|      American Woman|      The Guess Who|Mmm, da da da. Mm...|  1970|  1970-3|http://lyrics.wik...|Mmm, da da da.

##Pipeline Using Spark
Reference [combine all features into a single feature vector](https://databricks.com/blog/2015/07/29/new-features-in-machine-learning-pipelines-in-spark-1-4.html)
![Ensemble Pipeline Overview](https://databricks.com/wp-content/uploads/2015/07/simple-pipeline.png)
* Tokenizer
* HashingTF
* Word2Vec
* OneHotEncoder
* Vector Assembler

In [34]:
print "execution start --> {}".format(time.strftime('%a, %d %b %Y %H:%M:%S', time.localtime()))

execution start --> Fri, 27 Nov 2015 04:48:37


In [35]:
%%time
# Pipeline adapted from:
# http://spark.apache.org/docs/latest/ml-guide.html
# https://databricks.com/blog/2015/07/29/new-features-in-machine-learning-pipelines-in-spark-1-4.html
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row

# Prepare training documents from a list of (id, text, label) tuples.
LabeledDocument = Row("song_key", "lyrics", "label")

# training on songs to 2013
training = lyricsdf.filter(lyricsdf['year'] != 2014).select(['song_key','lyrics','label'])

## ML Pipeline 
tok = Tokenizer(inputCol="lyrics", outputCol="words")
htf = HashingTF(inputCol=tok.getOutputCol(), outputCol="features", numFeatures=200)

## path1 (need to work on input using corpus)
# w2v = Word2Vec(inputCol="lyrics", outputCol="w2v")
# ohe = OneHotEncoder(inputCol="label", outputCol="lbl")
# va = VectorAssembler(inputCols=["tf", "w2v", "lbl"], outputCol="features")
# pipeline = Pipeline(stages=[tok,htf,w2v,ohe,va])

## path2: three stages: tokenizer, hashingTF, and lr.
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tok, htf, lr])

# Fit the pipeline to training documents.
model = pipeline.fit(training)

CPU times: user 16.7 ms, sys: 33.9 ms, total: 50.6 ms
Wall time: 3.62 s


In [36]:
# test year 2014
test = lyricsdf.filter(lyricsdf['year'] == 2014).select(['song_key','lyrics','label'])

# Make predictions on test documents and print columns of interest.
prediction = model.transform(test)
selected = prediction.select("song_key", "lyrics", "prediction")

In [37]:
print type(selected)

<class 'pyspark.sql.dataframe.DataFrame'>


In [41]:
# build up predictions
pred_hits = {}
for row in selected.collect():
    pred_hits[row[0]] = row[2]

In [42]:
# quick check, predicted 2010-1 in top 25 correctly.
pred_hits['2014-1']

0.0

##How did we do at predicting top 50 hits for 2014 in light of hits prior?

In [47]:
hits = 0
misses = 0

for r in lyrics_pd_df[lyrics_pd_df['year'] == 2014].iterrows():
    song_key = r[1].song_key
    pred = pred_hits[song_key]  
    result = labelForPosition(r[1].position)
    correct = result == pred
    if correct:
        hits +=1
        print "Correct   ::: song_key --> {}, predicted top 25? {}".format(song_key, pred)
    else:
        misses +=1
        print "Incorrect ::: song_key --> {}, predicted top 25? {}".format(song_key, pred)

Correct   ::: song_key --> 2014-1, predicted top 25? 0.0
Correct   ::: song_key --> 2014-2, predicted top 25? 0.0
Correct   ::: song_key --> 2014-3, predicted top 25? 0.0
Incorrect ::: song_key --> 2014-4, predicted top 25? 1.0
Incorrect ::: song_key --> 2014-5, predicted top 25? 1.0
Correct   ::: song_key --> 2014-6, predicted top 25? 0.0
Correct   ::: song_key --> 2014-7, predicted top 25? 0.0
Incorrect ::: song_key --> 2014-8, predicted top 25? 1.0
Correct   ::: song_key --> 2014-9, predicted top 25? 0.0
Correct   ::: song_key --> 2014-10, predicted top 25? 0.0
Correct   ::: song_key --> 2014-11, predicted top 25? 0.0
Correct   ::: song_key --> 2014-12, predicted top 25? 0.0
Correct   ::: song_key --> 2014-13, predicted top 25? 0.0
Incorrect ::: song_key --> 2014-14, predicted top 25? 1.0
Incorrect ::: song_key --> 2014-15, predicted top 25? 1.0
Correct   ::: song_key --> 2014-16, predicted top 25? 0.0
Correct   ::: song_key --> 2014-17, predicted top 25? 0.0
Incorrect ::: song_key 

In [46]:
print "hits: {}, misses: {}".format(hits,misses)

hits: 59, misses: 40
